In [1]:
from func_def import *

In [19]:
df = pd.read_csv('data_pre_processed.csv',index_col=[0]).reset_index(drop=True)
df.tail(3)

,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_str,f_body_str,f_leg_str,f_distance_str,f_clinch_str,f_ground_str,o_head_str,o_body_str,o_leg_str,o_distance_str,o_clinch_str,o_ground_str,f_head_str_perc,f_body_str_perc,f_leg_str_perc,f_dist_str_perc,f_clinch_str_perc,f_ground_str_perc,o_head_str_perc,o_body_str_perc,o_leg_str_perc,o_dist_str_perc,o_clinch_str_perc,o_ground_str_perc
7236,http://www.ufcstats.com/fight-details/ccee020b...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Johnny Rhode,David Levick,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,9of15,1of1,1of1,1of1,1of1,9of15,4of5,0of0,0of0,1of2,2of2,1of1,0.81,0.09,0.09,0.09,0.09,0.81,1.0,0.0,0.0,0.25,0.5,0.25
7237,http://www.ufcstats.com/fight-details/4b9ae533...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Patrick Smit,Ray Wizar,Open Weight Bout,Submission,1,0:58,No Time Limit,0of0,1of1,0of0,0of0,1of1,0of0,0of0,0of0,1of1,1of1,0of0,0of0,0.00,1.00,0.00,0.00,1.00,0.00,0.0,0.0,1.0,1.00,0.0,0.00
7238,http://www.ufcstats.com/fight-details/4acab678...,http://www.ufcstats.com/event-details/a6a9ab5a...,1994-03-11,W,Scott Morri,Sean Daughert,Open Weight Bout,Submission,1,0:20,No Time Limit,1of1,0of0,0of0,0of0,1of1,0of0,0of2,0of0,0of2,0of3,0of1,0of0,1.00,0.00,0.00,0.00,1.00,0.00,0.0,0.0,0.0,0.00,0.0,0.00


In [20]:
# Extract features
FEATURES = [item[item.find('_')+1:] for item in df.columns if 'f_' in item]

FULL_FEATURE_NAME = [f'f_{item}' for item in FEATURES] + [f'o_{item}' for item in FEATURES]
print(f'Constructing Features: {FEATURES}')

drop_columns = []
for column in FULL_FEATURE_NAME:
    if type(df.loc[0,column]) == str:
        drop_columns.append(column)
        df[f'{column}_succ'] = pd.to_numeric([item.split('of')[0] for item in df[column]])
        df[f'{column}_att'] = pd.to_numeric([item.split('of')[1] for item in df[column]])

df = df[[column for column in df.columns if column not in drop_columns]]
df.head(3)

Constructing Features: ['head_str', 'body_str', 'leg_str', 'distance_str', 'clinch_str', 'ground_str', 'head_str_perc', 'body_str_perc', 'leg_str_perc', 'dist_str_perc', 'clinch_str_perc', 'ground_str_perc']


,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_str_perc,f_body_str_perc,f_leg_str_perc,f_dist_str_perc,f_clinch_str_perc,f_ground_str_perc,o_head_str_perc,o_body_str_perc,o_leg_str_perc,o_dist_str_perc,o_clinch_str_perc,o_ground_str_perc,f_head_str_succ,f_head_str_att,f_body_str_succ,f_body_str_att,f_leg_str_succ,f_leg_str_att,f_distance_str_succ,f_distance_str_att,f_clinch_str_succ,f_clinch_str_att,f_ground_str_succ,f_ground_str_att,o_head_str_succ,o_head_str_att,o_body_str_succ,o_body_str_att,o_leg_str_succ,o_leg_str_att,o_distance_str_succ,o_distance_str_att,o_clinch_str_succ,o_clinch_str_att,o_ground_str_succ,o_ground_str_att
0,http://www.ufcstats.com/fight-details/1aa4c2fa...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,L,Grant Dawson,Bobby Green,Lightweight Bout,KO/TKO,1,0:33,5 Rnd (5-5-5-5-5),0.33,0.34,0.33,0.33,0.34,0.33,1.00,0.00,0.00,0.14,0.00,0.85,0,6,0,0,0,0,0,6,0,0,0,0,7,10,0,0,0,1,1,4,0,0,6,7
1,http://www.ufcstats.com/fight-details/e966de63...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,W,Joe Pyfer,Abdul Razak Alhassa,Middleweight Bout,Submission,2,2:05,3 Rnd (5-5-5),0.57,0.34,0.07,0.92,0.07,0.00,0.36,0.18,0.45,1.00,0.00,0.00,15,47,9,12,2,3,24,60,2,2,0,0,4,15,2,3,5,10,11,27,0,1,0,0
2,http://www.ufcstats.com/fight-details/eb76a5b7...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,L,Alex Morono,Joaquin Buckley,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),0.72,0.25,0.02,0.92,0.07,0.00,0.60,0.35,0.04,0.80,0.02,0.16,29,122,10,17,1,2,37,133,3,8,0,0,41,134,24,36,3,6,55,141,2,9,11,26


In [21]:
# data enrichments
NEW_DATA_COLUMNS = ['win_perc','current_win_streak']
for column in NEW_DATA_COLUMNS:
    if column == 'win_perc':
        df[f'f_{column}'] = [get_win_perc(fighter=df.loc[i,'fighter'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]
        df[f'o_{column}'] = [get_win_perc(fighter=df.loc[i,'opponent'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]
    if column == 'current_win_streak':    
        df[f'f_{column}'] = [get_win_streak(fighter=df.loc[i,'fighter'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]
        df[f'o_{column}'] = [get_win_streak(fighter=df.loc[i,'opponent'],time=df.loc[i,'date'],data=df) for i in tqdm(range(len(df)))]

drop_columns = []
for feature in NEW_DATA_COLUMNS:
    df[f'{feature}_dif'] = df[f'f_{feature}'] - df[f'o_{feature}']
    drop_columns += [f'f_{feature}',f'o_{feature}']

df = df[[column for column in df.columns if column not in drop_columns]]
df.head(3)       

100%|██████████| 7239/7239 [00:25<00:00, 287.74it/s]


,url,event_url,date,result,fighter,opponent,title,method,round,time,format,f_head_str_perc,f_body_str_perc,f_leg_str_perc,f_dist_str_perc,f_clinch_str_perc,f_ground_str_perc,o_head_str_perc,o_body_str_perc,o_leg_str_perc,o_dist_str_perc,o_clinch_str_perc,o_ground_str_perc,f_head_str_succ,f_head_str_att,f_body_str_succ,f_body_str_att,f_leg_str_succ,f_leg_str_att,f_distance_str_succ,f_distance_str_att,f_clinch_str_succ,f_clinch_str_att,f_ground_str_succ,f_ground_str_att,o_head_str_succ,o_head_str_att,o_body_str_succ,o_body_str_att,o_leg_str_succ,o_leg_str_att,o_distance_str_succ,o_distance_str_att,o_clinch_str_succ,o_clinch_str_att,o_ground_str_succ,o_ground_str_att,win_perc_dif,current_win_streak_dif
0,http://www.ufcstats.com/fight-details/1aa4c2fa...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,L,Grant Dawson,Bobby Green,Lightweight Bout,KO/TKO,1,0:33,5 Rnd (5-5-5-5-5),0.33,0.34,0.33,0.33,0.34,0.33,1.00,0.00,0.00,0.14,0.00,0.85,0,6,0,0,0,0,0,6,0,0,0,0,7,10,0,0,0,1,1,4,0,0,6,7,0.375000,0
1,http://www.ufcstats.com/fight-details/e966de63...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,W,Joe Pyfer,Abdul Razak Alhassa,Middleweight Bout,Submission,2,2:05,3 Rnd (5-5-5),0.57,0.34,0.07,0.92,0.07,0.00,0.36,0.18,0.45,1.00,0.00,0.00,15,47,9,12,2,3,24,60,2,2,0,0,4,15,2,3,5,10,11,27,0,1,0,0,0.454545,0
2,http://www.ufcstats.com/fight-details/eb76a5b7...,http://www.ufcstats.com/event-details/c8a49ff2...,2023-10-07,L,Alex Morono,Joaquin Buckley,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),0.72,0.25,0.02,0.92,0.07,0.00,0.60,0.35,0.04,0.80,0.02,0.16,29,122,10,17,1,2,37,133,3,8,0,0,41,134,24,36,3,6,55,141,2,9,11,26,0.066667,0


In [22]:
df.current_win_streak_dif.unique()

array([  0,  -5,   1,   3,  -1,  -2,  -3,   5,   2,   4,  -7,  -4,   6,
        -8,  10,  -6,   7,   8, -12,  12,   9, -10,  13,  11, -20,  17,
       -11,  -9, -18, -14, -15, -17,  20,  16,  14,  19,  18, -13, -22,
        22,  21, -16])

In [7]:
N = 3 #recent fights to include for certain stats
STATS = [item for item in FEATURES] + [f'{item}_att' for item in FEATURES]

print(f'Preparing {STATS} statistics:')
for stat in STATS:
    data[f'f_{stat}_mean'] = [get_mean_stat(fighter=data.loc[i,'fighter'],stat_col=f'f_{stat}',time=data.loc[i,'date'],data=data) for i in tqdm(range(len(data)))]
    data[f'o_{stat}_mean'] = [get_mean_stat(fighter=data.loc[i,'opponent'],stat_col=f'o_{stat}',time=data.loc[i,'date'],data=data) for i in tqdm(range(len(data)))]
    data[f'f_{stat}_last_{N}_mean'] = [get_last_n_stat_mean(fighter=data.loc[i,'fighter'],stat_col=f'f_{stat}',time=data.loc[i,'date'],data=data,n=N) for i in tqdm(range(len(data)))]
    data[f'o_{stat}_last_{N}_mean'] = [get_last_n_stat_mean(fighter=data.loc[i,'opponent'],stat_col=f'o_{stat}',time=data.loc[i,'date'],data=data,n=N) for i in tqdm(range(len(data)))]

data.tail(3)

Preparing ['head_sig_str', 'body_sig_str', 'ground_sig_str', 'head_sig_str_att', 'body_sig_str_att', 'ground_sig_str_att'] statistics:


100%|██████████| 7195/7195 [00:20<00:00, 349.58it/s]


,date,fighter,opponent,result,f_head_sig_str_mean,o_head_sig_str_mean,f_head_sig_str_last_3_mean,o_head_sig_str_last_3_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_body_sig_str_last_3_mean,o_body_sig_str_last_3_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_head_sig_str_att_last_3_mean,o_head_sig_str_att_last_3_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_body_sig_str_att_last_3_mean,o_body_sig_str_att_last_3_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean
7192,1994-03-11,Johnny Rhode,David Levick,W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7193,1994-03-11,Patrick Smit,Ray Wizar,W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7194,1994-03-11,Scott Morri,Sean Daughert,W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
FEATURES

['head_sig_str', 'body_sig_str', 'ground_sig_str']

In [17]:
data.columns

Index(['date', 'fighter', 'opponent', 'result', 'f_head_sig_str_mean',
       'o_head_sig_str_mean', 'f_head_sig_str_last_3_mean',
       'o_head_sig_str_last_3_mean', 'f_body_sig_str_mean',
       'o_body_sig_str_mean', 'f_body_sig_str_last_3_mean',
       'o_body_sig_str_last_3_mean', 'f_ground_sig_str_mean',
       'o_ground_sig_str_mean', 'f_ground_sig_str_last_3_mean',
       'o_ground_sig_str_last_3_mean', 'f_head_sig_str_att_mean',
       'o_head_sig_str_att_mean', 'f_head_sig_str_att_last_3_mean',
       'o_head_sig_str_att_last_3_mean', 'f_body_sig_str_att_mean',
       'o_body_sig_str_att_mean', 'f_body_sig_str_att_last_3_mean',
       'o_body_sig_str_att_last_3_mean', 'f_ground_sig_str_att_mean',
       'o_ground_sig_str_att_mean', 'f_ground_sig_str_att_last_3_mean',
       'o_ground_sig_str_att_last_3_mean'],
      dtype='object')

In [18]:
for feature in FEATURES:
    data[f'{feature}_mean_dif'] = data[f'f_{feature}_mean'] - data[f'o_{feature}_mean']
    data[f'{feature}_att_mean_dif'] = data[f'f_{feature}_att_mean'] - data[f'o_{feature}_att_mean']
    data[f'{feature}_last_{N}_mean_dif'] = data[f'f_{feature}_last_{N}_mean'] - data[f'o_{feature}_last_{N}_mean']

for feature in NEW_DATA_COLUMNS:
    data[f'{feature}_dif'] = data[f'f_{feature}'] - data[f'o_{feature}']

data = data[[column for column in data.columns if 'dif' in column]] #include only dif columns
data.head(3)

KeyError: 'f_win_perc'

In [13]:
[column for column in data.columns if 'dif' in column]

[]

In [8]:
data.describe()

,f_head_sig_str_mean,o_head_sig_str_mean,f_head_sig_str_last_3_mean,o_head_sig_str_last_3_mean,f_body_sig_str_mean,o_body_sig_str_mean,f_body_sig_str_last_3_mean,o_body_sig_str_last_3_mean,f_ground_sig_str_mean,o_ground_sig_str_mean,f_ground_sig_str_last_3_mean,o_ground_sig_str_last_3_mean,f_head_sig_str_att_mean,o_head_sig_str_att_mean,f_head_sig_str_att_last_3_mean,o_head_sig_str_att_last_3_mean,f_body_sig_str_att_mean,o_body_sig_str_att_mean,f_body_sig_str_att_last_3_mean,o_body_sig_str_att_last_3_mean,f_ground_sig_str_att_mean,o_ground_sig_str_att_mean,f_ground_sig_str_att_last_3_mean,o_ground_sig_str_att_last_3_mean
count,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000,7195.000000
mean,20.394307,17.290891,21.364536,17.963238,6.290878,5.481619,6.729326,5.736484,5.450917,4.335557,5.114825,4.175075,55.633164,47.512469,58.685268,49.311119,9.056527,7.845728,9.726152,8.233218,8.095629,6.438413,7.471786,6.130577
std,14.508226,15.387917,17.298137,17.326087,5.502293,5.834125,6.532947,6.514428,5.837535,5.493821,6.684985,5.930990,39.013389,41.822428,45.836909,46.257928,7.785248,8.139487,9.164594,9.117152,8.797615,8.202162,9.835832,8.735805
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,2.000000,9.500000,2.000000,2.250000,0.000000,2.000000,0.000000,1.000000,0.000000,0.333333,0.000000,29.000000,7.000000,26.000000,6.666667,3.343137,0.500000,3.000000,0.333333,1.857143,0.000000,0.666667,0.000000
50%,19.000000,16.000000,18.666667,15.000000,5.250000,4.142857,5.333333,4.000000,4.000000,2.727273,3.000000,2.000000,51.800000,43.500000,52.333333,41.666667,7.625000,6.000000,7.666667,6.000000,6.000000,4.000000,4.000000,2.666667
75%,28.550505,26.000000,29.833333,27.000000,9.000000,8.250000,9.666667,8.666667,7.666667,6.500000,7.000000,6.000000,78.380000,72.750000,82.666667,75.333333,13.000000,12.000000,14.000000,12.500000,11.142857,9.666667,10.333333,9.000000
max,167.000000,138.000000,173.333333,149.000000,54.000000,64.000000,75.000000,70.000000,68.000000,57.000000,71.333333,57.000000,328.000000,397.000000,391.000000,397.000000,71.000000,83.000000,91.666667,87.333333,141.000000,88.000000,141.000000,88.000000
